In [50]:
import numpy as np
# load data
V = np.fromfile('../data/Verts')[2:]
D = np.fromfile('../data/exact')[2:]

V.resize(1511, 37500)
D.resize(1511, 26)

In [51]:
#PCA 
from sklearn.decomposition import PCA
pca = PCA(n_components=36)
newV = pca.fit_transform(V) #(1511, 36)


In [75]:
#Dnn Regressor
import tensorflow as tf
import tensorflow.contrib.learn as skflow
## Data Loader
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(D, newV, test_size=0.1, random_state=2)

def train_input_fn(batch_size=1):
    return tf.estimator.inputs.numpy_input_fn(
                    x={'X':x_train.astype(np.float32)}, 
                    y=y_train.astype(np.float32),
                    batch_size=batch_size,
                    shuffle=True)

def test_input_fn(batch_size=1):
    return tf.estimator.inputs.numpy_input_fn(
                    x={'X':x_test.astype(np.float32)}, 
                    y=y_test.astype(np.float32),
                    batch_size=batch_size,
                    shuffle=True)

feature_columns = [tf.feature_column.numeric_column('X', shape=(26,))]

In [76]:
## Dense Neural Network


hidden_layers = [32, 64, 128, 256, 128, 64]
dropout = 0.0

MODEL_PATH='./DNNRegressor01/'
for hl in hidden_layers:
	MODEL_PATH += '%s_' % hl
MODEL_PATH += 'D0%s' % (int(dropout*10))

validation_metrics = {"MSE": tf.contrib.metrics.streaming_mean_squared_error}


regressor = skflow.DNNRegressor(feature_columns=feature_columns,
                label_dimension=36,
                hidden_units=hidden_layers,
                model_dir=MODEL_PATH,
                dropout=dropout)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001D719DD59E8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './DNNRegressor01/32_64_128_256_128_64_D00'}


In [77]:
regressor.fit(input_fn=train_input_fn(batch_size=1), max_steps=10000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./DNNRegressor01/32_64_128_256_128_64_D00\model.ckpt.
INFO:tensorflow:loss = 1.2117652, step = 0
INFO:tensorflow:global_step/sec: 327.676
INFO:tensorflow:loss = 0.22569749, step = 100 (0.306 sec)
INFO:tensorflow:global_step/sec: 511.469
INFO:tensorflow:loss = 1.5606604, step = 200 (0.195 sec)
INFO:tensorflow:global_step/sec: 457.924
INFO:tensorflow:loss = 2.767291, step = 300 (0.218 sec)
INFO:tensorflow:global_step/sec: 439.282
INFO:tensorflow:loss = 0.87687945, step = 400 (0.227 sec)
INFO:tensorflow:global_step/sec: 485.369
INFO:tensorflow:loss = 0.5145762, step = 500 (0.207 sec)
INFO:tensorflow:global_step/sec: 455.691
INFO:tensorflow:loss = 1.1560766, step = 600 (0.219 sec)
INFO:tensorflow:global_step/sec: 498.516
INFO:tensorflow:loss = 0.34476975, step = 700 (0.201 sec)
INFO:

DNNRegressor(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._RegressionHead object at 0x000001D719DD5898>, 'hidden_units': [32, 64, 128, 256, 128, 64], 'feature_columns': (_NumericColumn(key='X', shape=(26,), default_value=None, dtype=tf.float32, normalizer_fn=None),), 'optimizer': None, 'activation_fn': <function relu at 0x000001D5202D4F28>, 'dropout': 0.0, 'gradient_clip_norm': None, 'embedding_lr_multipliers': None, 'input_layer_min_slice_size': None})

In [78]:
regressor.evaluate(input_fn=test_input_fn(batch_size=1), steps=1)

INFO:tensorflow:Starting evaluation at 2019-04-17-03:04:42
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNNRegressor01/32_64_128_256_128_64_D00\model.ckpt-1359
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2019-04-17-03:04:42
INFO:tensorflow:Saving dict for global step 1359: global_step = 1359, loss = 2.7148807


{'global_step': 1359, 'loss': 2.7148807}

In [86]:
data = np.asarray([[1.2, 1.2, 1.44258, 1.47699, 0.761712, 0.650765, 0.52607, 0.300625, 0.23843, 0.29884, 0.801796, 0.579924, 0.54716, 0.331767, 0.47362, 0.45998, 0.467646, 0.318187, 0.34539, 0.377015, 0.380852, 0.642929, 0.463352, 0.391945, 0.328395, 1.121300]])

pred = regressor.predict(x={'X':data}, as_iterable=False)

print(pred.shape)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNNRegressor01/32_64_128_256_128_64_D00\model.ckpt-1359
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
(1, 36)


In [87]:
out = np.dot(pred, pca.components_)
np.savetxt('../data/new.txt', out, delimiter=' ')